In [1]:
import torch
import torch.nn as nn
import torch.functional as F
import pytorch_lightning as pl

# from torchrbpnet.layers import Conv1DFirstLayer, Conv1DResBlock, IndexEmbeddingOutputHead
# from torchrbpnet.losses import MultinomialNLLLossFromLogits

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# %%
class Conv1DFirstLayer(nn.Module):
    def __init__(self, in_chan, filters=128, kernel_size=12):
        super(Conv1DFirstLayer, self).__init__()

        self.conv1d = nn.Conv1d(in_chan, filters, kernel_size=kernel_size, padding='same')
        self.act = nn.ReLU()
    
    def forward(self, inputs, **kwargs):
        x = self.conv1d(inputs)
        x = self.act(x)
        return x

# %%
class Conv1DResBlock(nn.Module):
    def __init__(self, in_chan, filters=128, kernel_size=3, dropout=0.25, dilation=1, residual=True):
        super(Conv1DResBlock, self).__init__()

        self.conv1d = nn.Conv1d(in_chan, filters, kernel_size=kernel_size, dilation=dilation, padding='same')
        self.batch_norm = nn.BatchNorm1d(filters)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.residual = residual
    
    def forward(self, inputs, **kwargs):
        x = self.conv1d(inputs)
        x = self.batch_norm(x)
        x = self.act(x)
        x = self.dropout(x)
        if self.residual:
            x = inputs + x
        return x

# %%
class IndexEmbeddingOutputHead(nn.Module):
    def __init__(self, n_tasks, dims):
        super(IndexEmbeddingOutputHead, self).__init__()

        # protein/experiment embedding of shape (p, d)
        self.embedding = torch.nn.Embedding(n_tasks, dims)
    
    def forward(self, bottleneck, **kwargs):
        # bottleneck of shape (batch, d, n) --> (batch, n, d)
        bottleneck = torch.transpose(bottleneck, -1, -2)
        
        # embedding of (batch, p, d) --> (batch, d, p)
        embedding = torch.transpose(self.embedding.weight, 0, 1)

        logits = torch.matmul(bottleneck, embedding) # torch.transpose(self.embedding.weight, 0, 1)  
        return logits

In [20]:
class Network(nn.Module):
    def __init__(self, tasks, nlayers=9):
        super(Network, self).__init__()

        self.tasks = tasks

        self.body = nn.Sequential(*[Conv1DFirstLayer(4, 128)]+[(Conv1DResBlock(128, dilation=(2**i))) for i in range(nlayers)])
        self.head = IndexEmbeddingOutputHead(len(self.tasks), dims=128)
    
    def forward(self, inputs, **kwargs):
        x = inputs

        for layer in self.body:
            x = layer(x)

        return self.head(x)
network = Network(tasks=list(range(223)))

In [5]:
from bioflow import io
import tensorflow as tf

class TFIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, filepath, features_filepath=None, batch_size=64, cache=True, shuffle=None):
        super(TFIterableDataset).__init__()

        self.dataset = io.dataset_ops.load_tfrecord(filepath, deserialize=False)

        # cache
        if cache:
            self.dataset = self.dataset.cache()

        if shuffle:
            self.dataset = self.dataset.shuffle(shuffle)

        # deserialize
        if features_filepath is None:
            features_filepath = filepath + '.features.json'
        self.features = io.dataset_ops.features_from_json_file(features_filepath)
        self.dataset = io.dataset_ops.deserialize_dataset(self.dataset, self.features)

        # batch
        self.dataset = self.dataset.batch(batch_size)

        # format dataset
        self.dataset = self.dataset.map(lambda e: (tf.transpose(e['inputs']['input'], perm=[0, 2, 1]), tf.transpose(e['outputs']['signal']['total'], perm=[0, 2, 1])))
        
    def __iter__(self):
        for example in self.dataset.as_numpy_iterator():
            yield tf.nest.map_structure(lambda x: torch.tensor(x).to(torch.float32), example)

dataset = TFIterableDataset('example-data-matrix/windows.chr13.4.data.matrix.filtered.tfrecord', shuffle=1_000_000)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=None)
example = next(iter(dataloader))
print(example[0].shape)
print(example[1].shape)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
torch.Size([64, 4, 1000])
torch.Size([64, 1000, 223])


In [18]:
class MultinomialNLLLossFromLogits(nn.Module):
    def __init__(self, reduction=torch.mean):
        super(MultinomialNLLLossFromLogits, self).__init__()
        self.reduction = reduction
    
    def __call__(self, y, y_pred, dim=-1):
        neg_log_probs = self.log_likelihood_from_logits(y, y_pred, dim) * -1
        if self.reduction is not None:
            return self.reduction(neg_log_probs)
        return neg_log_probs

    def log_likelihood_from_logits(self, y, y_pred, dim):
        return torch.sum(torch.mul(torch.log_softmax(y_pred, dim=dim), y), dim=dim) + self.log_combinations(y, dim)

    def log_combinations(self, input, dim):
        total_permutations = torch.lgamma(torch.sum(input, dim=dim) + 1)
        counts_factorial = torch.lgamma(input + 1)
        redundant_permutations = torch.sum(counts_factorial, dim=dim)
        return total_permutations - redundant_permutations

In [21]:
y = example[1]
print(y.shape)
y_pred = network(example[0])
print(y_pred.shape)

loss_fn = MultinomialNLLLossFromLogits()
loss_fn(y, y_pred, dim=-2)

torch.Size([64, 1000, 223])
torch.Size([64, 1000, 223])


tensor(194.2709, grad_fn=<MeanBackward0>)

In [10]:
class MultiRBPNet(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.network = Network(tasks=range(223))
        self.loss_fn = MultinomialNLLLossFromLogits()

    def training_step(self, batch, **kwargs):
        x, y = batch
        y_pred = self.network(x)
        loss = self.loss_fn(y, y_pred, dim=-2)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

model = MultiRBPNet()

In [11]:
trainer = pl.Trainer(max_epochs=2)
trainer.fit(model=model, train_dataloaders=dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name    | Type                         | Params
---------------------------------------------------------
0 | network | Network                      | 481 K 
1 | loss_fn | MultinomialNLLLossFromLogits | 0     
---------------------------------------------------------
481 K     Trainable params
0         Non-trainable params
481 K     Total params
1.928     Total estimated model params size (MB)
/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many 

Epoch 0: : 0it [00:00, ?it/s]

RuntimeError: The size of tensor a (233) must match the size of tensor b (223) at non-singleton dimension 2